<a href="https://colab.research.google.com/github/sathyanarayanajammala/GenAI/blob/main/Langchain_Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative AI and Prompt Engineering
## A Program by IISc and TalentSprint



## **LangChain: Routing**

### **[Reference](https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/)**

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain-openai

In [ ]:
import openai
import os

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read().strip()          # Remove Blank Spaces
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
prompt_classify = PromptTemplate.from_template("""Given the user question below, classify it as either being about
 `Physics`, `Math`, `Computer Science`.Do not respond with more than one word.
 <question> {question} </question> Classification:""")


In [ ]:
prompt_classify

In [ ]:
chain = prompt_classify  | llm | StrOutputParser()

chain.invoke({'question':"what is fisrt law of motion?"})

In [ ]:
chain.invoke({"question":"What's a programming language?"})

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{question}
Start your answer with `Subject:Physics :`
"""
physics_prompt=ChatPromptTemplate.from_template(physics_template) # ChatPromptTemplate , PromptTemplate
physics_chain = physics_prompt | llm | StrOutputParser()

In [ ]:
math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{question}
Start your answer with `Subject:Math :`
"""
math_prompt=ChatPromptTemplate.from_template(math_template)
math_chain = math_prompt | llm | StrOutputParser()

In [ ]:
computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{question}
Start your answer with `Subject:Computer Science :`
"""

computer_science_prompt=ChatPromptTemplate.from_template(computerscience_template)
computer_science_chain = computer_science_prompt | llm | StrOutputParser()

In [ ]:
 general_prompt=ChatPromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Start your answer with `Subject:General :`"""

)
 general_chain = general_prompt | llm | StrOutputParser()

In [ ]:
def route(info):
    if "physics" in info["topic"].lower():
        return physics_chain
    elif "math" in info["topic"].lower():
        return math_chain
    elif "computer science" in info["topic"].lower():
        return computer_science_chain
    else:
        return general_chain

In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)
full_chain

In [ ]:
full_chain.invoke({"question":"What is black body radiation?"})

In [ ]:
full_chain.invoke({"question":"What's a programming language?"})

In [ ]:
full_chain.invoke({"question":"What's 1+1"})

In [ ]:
full_chain.invoke({"question":"What's a path integral?"})

In [ ]:
full_chain.invoke({"question":"tell me 2 facts about Taj Mahal India?"})

#**Legacy**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

In [ ]:
llm_model = "gpt-3.5-turbo" # This is a chat model
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "history",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain=LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
#router_template

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [ ]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.invoke("What is black body radiation?")